In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [2]:
import evaluate
import numpy as np
import torch
from torch import nn
import copy

from transformers import (
    pipeline,
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    default_data_collator,
    Seq2SeqTrainingArguments,
    GenerationConfig,
    AdamW,
    get_scheduler
)

from transformers.models.t5.modeling_t5 import (
    T5Stack,
    T5Block,
    T5LayerNorm,
    T5Config
)

from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
# load the model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

# Summarization

In [5]:
def summarization(dataset, model):

    prompt = 'Summarize the following:\n[PASSAGE]\n.'
    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]
    # print(example.keys())

    context, summary = example['document'], example['summary']

    prompt = prompt.replace("[PASSAGE]", context)
    input_ids = tokenizer(prompt, max_length=1024, truncation=True, return_tensors="pt").input_ids.to(device)

    generation_config = GenerationConfig(early_stopping=False, length_penalty= 0.0, early_stop=False, max_length=150)

    outputs = model.generate(input_ids, generation_config=generation_config)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    print("Input:\n", prompt)
    print("Predicted Summary:", result)
    print("Reference Summary:", summary)

dataset = load_dataset("multi_news", "3.0.0", split='validation')
summarization(dataset, model)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Input:
 Summarize the following:
SAN FRANCISCO — Authorities on Tuesday took a 17-year-old into custody after he opened the emergency exit of a jet preparing to taxi at San Francisco International Airport, jumped onto the wing and dropped down to the tarmac, an airport spokesman said. 
 
 The bizarre incident unfolded at about 2:30 p.m. as Copa Airlines Flight 208 from Panama City was waiting to taxi to its arrival gate, according to a statement released by SFO spokesman Doug Yakel. 
 
 “Apparently he slid down the wing onto the tarmac,” said Yakel, adding that the boy was not injured. 
 
 An airfield construction crew confronted the teen and held him until San Francisco police arrived to take him into custody. He was later handed over to U.S. Customs and Border Protection, Yakel said. 
 
 The boy was taken to a hospital for a medical evaluation, Yakel said Wednesday. Related Articles Teen evaluated after jumping from plane’s emergency exit when it landed at SFO 
 
 The boy was not ide

In [6]:
def summarization_with_pipeline(dataset):

    summarization_pipeline = pipeline("summarization", model="google/flan-t5-small")

    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]
    # print(example.keys())

    context, summary = example['document'], example['summary']

    predicted_summary = summarization_pipeline(context, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, no_repeat_ngram_size=2)
    result = predicted_summary[0]['summary_text']

    print("Predicted Summary:", result)
    print("Reference Summary:", summary)

dataset = load_dataset("multi_news", "3.0.0", split='validation')
summarization_with_pipeline(dataset)

Token indices sequence length is longer than the specified maximum sequence length for this model (2442 > 512). Running this sequence through the model will result in indexing errors


Predicted Summary: – It's not a terrible, terrible movie, but it is good. The trailer manages to give away the whole movie without telling you how the story is told, and I'm not surprised. I don't think it has to do with the quality of the film so much as the premise. "Knocked Up," and everyone else gets to play...a fun character." Then someone told her she was funny. And she keeps making comedies now where the movie should start, once it does kick in to the main story. Spencer (Ashton Kutcher) finally decides at the end of that first 25 minutes that he’s in love with Heigl
Reference Summary: – The makers of Killers, the new rom-com with Katherine Heigl and Ashton Kutcher, didn't let critics screen it. Always a great sign. From the few reviews out there: Geoff Berkshire, Metromix: "Utterly unconvincing as action, comedy or romance, it’s a colossal waste of time and a shocking choice for Heigl at this point in her career." Drew McWeeny, HitFix: "This is not a terrible, terrible, terribl

# Q&A

In [7]:
def q_n_a(dataset, model):

    prompt = 'Given the following:\n[PASSAGE].\nAnswer the following:\n[QUESTION].\n'
    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]

    context, question, actual_answer = example['context'],  example['question'], example["answers"]

    prompt = prompt.replace("[PASSAGE]", context)
    prompt = prompt.replace("[QUESTION]", question)
    input_ids = tokenizer(prompt, max_length=512, truncation=True, return_tensors="pt").input_ids.to(device)

    outputs = model.generate(input_ids)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    print("Input:\n", prompt)
    print("Predicted Answer:", result)
    print("Reference Answers:", actual_answer)

dataset_sqaud = load_dataset("squad", split='validation')
q_n_a(dataset_sqaud, model)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input:
 Given the following:
Free movement of goods within the European Union is achieved by a customs union, and the principle of non-discrimination. The EU manages imports from non-member states, duties between member states are prohibited, and imports circulate freely. In addition under the Treaty on the Functioning of the European Union article 34, ‘Quantitative restrictions on imports and all measures having equivalent effect shall be prohibited between Member States’. In Procureur du Roi v Dassonville the Court of Justice held that this rule meant all "trading rules" that are "enacted by Member States" which could hinder trade "directly or indirectly, actually or potentially" would be caught by article 34. This meant that a Belgian law requiring Scotch whisky imports to have a certificate of origin was unlikely to be lawful. It discriminated against parallel importers like Mr Dassonville, who could not get certificates from authorities in France, where they bought the Scotch. Thi

In [8]:
def q_n_a_with_pipeline(dataset):

    qa_pipeline = pipeline("question-answering", model="google/flan-t5-small")

    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]

    context, question, actual_answer = example['context'],  example['question'], example["answers"]

    answer = qa_pipeline(question=question, context=context)
    result = answer["answer"]

    print("Predicted Answer:", result)
    print("Reference Answers:", actual_answer)
    print("Confidence:", answer["score"])

dataset_sqaud = load_dataset("squad", split='validation')
q_n_a_with_pipeline(dataset_sqaud)

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Answer:  publicity. Even at the draft stage, this finding was disputed
Reference Answers: {'text': ['18 July 2000', 'May 2000', '18 July 2000'], 'answer_start': [560, 272, 560]}
Confidence: 1.820284887799062e-05


# English to French Translation

In [9]:
def english_to_french_translation(dataset, model):

    prompt = 'Translate this sentence from English to French :\n[SENTENCE]\n.'
    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]

    english, french = example['en'], example['fr']

    prompt = prompt.replace("[SENTENCE]", english)
    input_ids = tokenizer(prompt, max_length=512, truncation=True, return_tensors="pt").input_ids.to(device)

    outputs = model.generate(input_ids)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    print("Input:\n", prompt)
    print("Predicted Translation:", result)
    print("Reference Translation:", french)

translation_dataset = load_dataset("enimai/MuST-C-fr", split='validation')
english_to_french_translation(translation_dataset, model)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input:
 Translate this sentence from English to French :
We went back to the Titanic wreck.
.
Predicted Translation: ['Nous avons savons à la wrecke de Titanic.']
Reference Translation: Nous sommes retournés à l'épave du Titanic.


In [10]:
def english_to_french_translation_with_pipeline(dataset):

    translation_pipeline = pipeline("translation_en_to_fr", model="google/flan-t5-small")

    example = dataset[int(np.random.randint(len(dataset), size=1)[0])]

    english, french = example['en'], example['fr']

    french_translation = translation_pipeline(english)
    result = french_translation[0]["translation_text"]

    print("Predicted Translation:", result)
    print("Reference Translation:", french)

translation_dataset = load_dataset("enimai/MuST-C-fr", split='validation')
english_to_french_translation_with_pipeline(translation_dataset)

Predicted Translation: Mais si vous immersez dans une adventure epice et vous tellez, « Nous avons l'élimination de l'huile.
Reference Translation: Mais si vous vous immergez dans une aventure héroïque et entendez : "Nous n'avons plus d'essence." Cela devient une histoire extraordinaire et une aventure à vivre.


# Names of all the model layers and their dimensions.

In [11]:
for name, param in model.named_parameters():
    if 'weight' in name:  # Print only for weight parameters
        print(f"Layer: {name} - Dimensions: {param.size()}")

Layer: shared.weight - Dimensions: torch.Size([32128, 512])
Layer: encoder.block.0.layer.0.SelfAttention.q.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.k.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.v.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.o.weight - Dimensions: torch.Size([512, 384])
Layer: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight - Dimensions: torch.Size([32, 6])
Layer: encoder.block.0.layer.0.layer_norm.weight - Dimensions: torch.Size([512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_0.weight - Dimensions: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_1.weight - Dimensions: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wo.weight - Dimensions: torch.Size([512, 1024])
Layer: encoder.block.0.layer.1.layer_norm.weight - Dimensions: torch.Size([512])
Layer: encoder.blo

# Total number of parameters/weights in this model.

In [12]:
# Get the total number of parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"Total Number of Parameters: {total_params}")


Total Number of Parameters: 76961152


# Final layer (decoder.final_layer_norm.weight) to all zeros and verification of Q&A task.

In [13]:
# Set the tensor in the final layer to all zeros
model.decoder.final_layer_norm.weight.data.fill_(0)

q_n_a(dataset_sqaud, model)

Input:
 Given the following:
Photorespiration can occur when the oxygen concentration is too high. Rubisco cannot distinguish between oxygen and carbon dioxide very well, so it can accidentally add O2 instead of CO2 to RuBP. This process reduces the efficiency of photosynthesis—it consumes ATP and oxygen, releases CO2, and produces no sugar. It can waste up to half the carbon fixed by the Calvin cycle. Several mechanisms have evolved in different lineages that raise the carbon dioxide concentration relative to oxygen within the chloroplast, increasing the efficiency of photosynthesis. These mechanisms are called carbon dioxide concentrating mechanisms, or CCMs. These include Crassulacean acid metabolism, C4 carbon fixation, and pyrenoids. Chloroplasts in C4 plants are notable as they exhibit a distinct chloroplast dimorphism..
Answer the following:
What is unusual about C4 plants' chloroplasts?.

Predicted Answer: ['']
Reference Answers: {'text': ['they exhibit a distinct chloroplast d

# Custom layer
Replacing the decoder.final_layer_norm.weight with a layer of smaller dimensions (128) and adjust all the dependent layers to match the dimension.

The Cell below replaces the final layer norm and dependent layers of dimension 512 with a dimension of size 128.

The class CustomLayerNorm adds a linear layer to map the input from 512 to 128 dimension. It then adds a LayerNorm of size 128.

The class CustomDecoder extends T5Stack and adds the CustomLayerNorm at the top. As the output from the CustomLayerNorm is of size 128, it adds another linear to map the input from 128 to the vocab_size of the model.

The class CustomT5 extends T5ForConditionalGeneration and combines the encoder with the CustomDecoder.

The class CustomDecoder only overrides the final layer norm and classCustomT5 only overrides the decoder. The rest of the functions of google/flan-t5-small remains same.

In [14]:
class CustomLayerNorm(nn.Module):
    def __init__(self, hidden_size, input_dim=512, eps=1e-6):
        """
        Construct a layernorm module in the T5 style. No bias and no subtraction of mean.
        """
        super().__init__()
        self.projection = nn.Linear(input_dim, hidden_size)
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        hidden_states = self.projection(hidden_states)

        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # convert into half-precision if necessary
        if self.weight.dtype in [torch.float16, torch.bfloat16]:
            hidden_states = hidden_states.to(self.weight.dtype)

        return self.weight * hidden_states

class CustomDecoder(T5Stack):
    def __init__(self, config, embed_tokens=None, old_dim=512, new_dim=128, vocab_size=31522):
        super().__init__(config, embed_tokens=embed_tokens)

        self.embed_tokens = embed_tokens
        self.is_decoder = config.is_decoder
        self.new_dim = new_dim

        self.block = nn.ModuleList(
            [T5Block(config, has_relative_attention_bias=bool(i == 0)) for i in range(config.num_layers)]
        )
        self.final_layer_norm = CustomLayerNorm(new_dim, input_dim=config.d_model, eps=config.layer_norm_epsilon)
        self.dropout = nn.Dropout(config.dropout_rate)

        # Initialize weights and apply final processing
        self.post_init()
        # Model parallel
        self.model_parallel = False
        self.device_map = None
        self.gradient_checkpointing = False
        # self.output_layer = nn.Linear(new_dim, vocab_size)

class CustomT5(T5ForConditionalGeneration):
    def __init__(self, config: T5Config):
        super().__init__(config)

        decoder_config = copy.deepcopy(config)
        decoder_config.is_decoder = True
        decoder_config.is_encoder_decoder = False
        decoder_config.num_layers = config.num_decoder_layers
        self.decoder = CustomDecoder(decoder_config, self.shared)

        if hasattr(self.decoder, 'new_dim'):
            new_dim = self.decoder.new_dim
        else:
            new_dim = decoder_config.d_model

        self.lm_head = nn.Linear(new_dim, config.vocab_size, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

In [15]:
custom_model = CustomT5(model.config)
for name, param in custom_model.named_parameters():
    if 'weight' in name:  # Print only for weight parameters
        print(f"Layer: {name} - Dimensions: {param.size()}")

Layer: shared.weight - Dimensions: torch.Size([32128, 512])
Layer: encoder.block.0.layer.0.SelfAttention.q.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.k.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.v.weight - Dimensions: torch.Size([384, 512])
Layer: encoder.block.0.layer.0.SelfAttention.o.weight - Dimensions: torch.Size([512, 384])
Layer: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight - Dimensions: torch.Size([32, 6])
Layer: encoder.block.0.layer.0.layer_norm.weight - Dimensions: torch.Size([512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_0.weight - Dimensions: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wi_1.weight - Dimensions: torch.Size([1024, 512])
Layer: encoder.block.0.layer.1.DenseReluDense.wo.weight - Dimensions: torch.Size([512, 1024])
Layer: encoder.block.0.layer.1.layer_norm.weight - Dimensions: torch.Size([512])
Layer: encoder.blo

In [16]:
custom_model.to(device)
q_n_a(dataset_sqaud, custom_model)

Input:
 Given the following:
The next major step occurred when James Watt developed (1763–1775) an improved version of Newcomen's engine, with a separate condenser. Boulton and Watt's early engines used half as much coal as John Smeaton's improved version of Newcomen's. Newcomen's and Watt's early engines were "atmospheric". They were powered by air pressure pushing a piston into the partial vacuum generated by condensing steam, instead of the pressure of expanding steam. The engine cylinders had to be large because the only usable force acting on them was due to atmospheric pressure..
Answer the following:
In an atmospheric engine, what does air pressure push against?.

Predicted Answer: ['assemblageabsorbed hitconceived verkaufen Nicoenberg Thousand77strandgehobenizărick Bahamas Benjamin jeans cabinCR cash']
Reference Answers: {'text': ['piston', 'a piston', 'piston', 'a piston'], 'answer_start': [342, 340, 342, 340]}


# Finetuning For SQUAD.

In [ ]:
dataset = load_dataset("squad")
print("Context: ", dataset["train"][0]["context"])
print("Question: ", dataset["train"][0]["question"])
print("Answer: ", dataset["train"][0]["answers"])


Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [ ]:
def preprocess_function(examples, padding="max_length", max_input_length=512, max_target_length=32):

    answers = [example['text'][0] for example in examples["answers"]]
    questions = [q.strip() for q in examples["question"]]
    contexts = [context.strip() for context in examples["context"]]

    assert len(questions) == len(contexts)
    prompts = [f'SQUAD Reading Comprehension Task\n\nGiven the following passage:\n{context}\nAnswer the question:\n{question}\n'
               for context, question in list(zip(contexts, questions))]

    inputs = tokenizer(
        prompts,
        max_length=max_input_length,
        truncation=True,
        padding=True,
    )
    labels = tokenizer(answers, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
# Metric
metric = evaluate.load("squad")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

def compute_metrics(preds, labels, valiation_set):
    final_preds = []

    for pred_list in preds:
        cur_pred = []
        for token in pred_list:
            if token == tokenizer.eos_token_id:
                break
            cur_pred.append(token)
        final_preds.append(cur_pred)
    decoded_preds = tokenizer.batch_decode(final_preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    for label_list in labels:
        for i in range(len(label_list)):
            if label_list[i] == -100:
                label_list[i] = tokenizer.pad_token_id
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    assert len(decoded_preds) == len(valiation_set)
    references = []
    final_preds = []

    for i, pred in enumerate(decoded_preds):
        final_preds.append({
            "id": validation_set[i]['id'],
            "prediction_text": pred
        })
        references.append({
            "id": validation_set[i]['id'],
            "answers": validation_set[i]['answers']
        })

    result = metric.compute(predictions=final_preds, references=references)
    print(result)
    return result

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
validation_set = [example for example in tokenized_dataset["validation"]]

In [ ]:
tokenized_dataset['train'] = tokenized_dataset['train'].remove_columns(["title","id", "context", "question", "answers"])
tokenized_dataset["validation"] = tokenized_dataset["validation"].remove_columns(["title", "id", "context", "question", "answers"])

In [ ]:
train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

10950


In [ ]:
def evaluate_model():
    model.eval()
    predictions = []
    labels = []
    print("Running Evaluation")
    for batch in tqdm(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        val_loss = outputs.loss
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.tolist())
        labels.extend(batch['labels'].tolist())
    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{num_training_steps}], Val_loss: {val_loss.item():.4f}")
    compute_metrics(predictions, labels, validation_set)
    model.train()

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
progress_bar = tqdm(range(num_training_steps))
step = 0
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        if step%100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{num_training_steps}], lr: {optimizer.param_groups[0]['lr']}, Loss: {loss.item()}")
        if step > 0 and step%1000 == 0:
            evaluate_model()
        step+=1

    model.save_pretrained(f"checkpoint_{epoch}")


  0%|          | 0/10950 [00:00<?, ?it/s]

Epoch [1/1], Step [1/10950], lr: 4.999543378995434e-05, Loss: 0.8829897046089172
Epoch [1/1], Step [101/10950], lr: 4.9538812785388126e-05, Loss: 0.5722970962524414
Epoch [1/1], Step [201/10950], lr: 4.908219178082192e-05, Loss: 0.7051485180854797
Epoch [1/1], Step [301/10950], lr: 4.862557077625571e-05, Loss: 0.4848741888999939
Epoch [1/1], Step [401/10950], lr: 4.8168949771689495e-05, Loss: 0.5393522381782532
Epoch [1/1], Step [501/10950], lr: 4.771232876712329e-05, Loss: 0.500650942325592
Epoch [1/1], Step [601/10950], lr: 4.725570776255708e-05, Loss: 0.401753306388855
Epoch [1/1], Step [701/10950], lr: 4.679908675799087e-05, Loss: 0.6067529916763306
Epoch [1/1], Step [801/10950], lr: 4.6342465753424656e-05, Loss: 0.8947744369506836
Epoch [1/1], Step [901/10950], lr: 4.588584474885845e-05, Loss: 0.7757418751716614
Epoch [1/1], Step [1001/10950], lr: 4.542922374429224e-05, Loss: 0.7994036674499512
Running Evaluation


  0%|          | 0/1322 [00:00<?, ?it/s]

Epoch [1/1], Step [1001/10950], Val_loss: 0.9706
{'exact_match': 62.157048249763484, 'f1': 83.30412306065195}
Epoch [1/1], Step [1101/10950], lr: 4.4972602739726025e-05, Loss: 0.3247883915901184
Epoch [1/1], Step [1201/10950], lr: 4.451598173515982e-05, Loss: 0.2604353725910187
Epoch [1/1], Step [1301/10950], lr: 4.405936073059361e-05, Loss: 0.6397036910057068
Epoch [1/1], Step [1401/10950], lr: 4.3602739726027394e-05, Loss: 0.25009840726852417
Epoch [1/1], Step [1501/10950], lr: 4.3146118721461186e-05, Loss: 0.885596513748169
Epoch [1/1], Step [1601/10950], lr: 4.268949771689498e-05, Loss: 0.25569847226142883
Epoch [1/1], Step [1701/10950], lr: 4.223287671232877e-05, Loss: 0.8788554668426514
Epoch [1/1], Step [1801/10950], lr: 4.1776255707762555e-05, Loss: 0.41101524233818054
Epoch [1/1], Step [1901/10950], lr: 4.131963470319635e-05, Loss: 0.5633679628372192
Epoch [1/1], Step [2001/10950], lr: 4.086301369863014e-05, Loss: 0.6801570653915405
Running Evaluation


  0%|          | 0/1322 [00:00<?, ?it/s]

...

Epoch [1/1], Step [8001/10950], Val_loss: 1.1790
{'exact_match': 63.9356669820246, 'f1': 84.21583618689448}
Epoch [1/1], Step [8101/10950], lr: 1.3009132420091325e-05, Loss: 0.47399070858955383
Epoch [1/1], Step [8201/10950], lr: 1.2552511415525115e-05, Loss: 0.5210881233215332
Epoch [1/1], Step [8301/10950], lr: 1.2095890410958905e-05, Loss: 0.2547300457954407
Epoch [1/1], Step [8401/10950], lr: 1.1639269406392694e-05, Loss: 0.6136345267295837
Epoch [1/1], Step [8501/10950], lr: 1.1182648401826484e-05, Loss: 0.6272262334823608
Epoch [1/1], Step [8601/10950], lr: 1.0726027397260274e-05, Loss: 0.5698612928390503
Epoch [1/1], Step [8701/10950], lr: 1.0269406392694065e-05, Loss: 0.25957202911376953
Epoch [1/1], Step [8801/10950], lr: 9.812785388127855e-06, Loss: 0.5838450193405151
Epoch [1/1], Step [8901/10950], lr: 9.356164383561643e-06, Loss: 0.28319817781448364
Epoch [1/1], Step [9001/10950], lr: 8.899543378995434e-06, Loss: 0.24632155895233154
Running Evaluation


  0%|          | 0/1322 [00:00<?, ?it/s]

Epoch [1/1], Step [9001/10950], Val_loss: 1.1111
{'exact_match': 63.35856196783349, 'f1': 83.96730084865327}
Epoch [1/1], Step [9101/10950], lr: 8.442922374429224e-06, Loss: 0.15208861231803894
Epoch [1/1], Step [9201/10950], lr: 7.986301369863014e-06, Loss: 0.3540201485157013
Epoch [1/1], Step [9301/10950], lr: 7.529680365296804e-06, Loss: 0.4582085907459259
Epoch [1/1], Step [9401/10950], lr: 7.073059360730594e-06, Loss: 0.49788564443588257
Epoch [1/1], Step [9501/10950], lr: 6.616438356164384e-06, Loss: 0.4667946696281433
Epoch [1/1], Step [9601/10950], lr: 6.159817351598174e-06, Loss: 0.976475715637207
Epoch [1/1], Step [9701/10950], lr: 5.703196347031964e-06, Loss: 1.163201093673706
Epoch [1/1], Step [9801/10950], lr: 5.246575342465753e-06, Loss: 0.2660379111766815
Epoch [1/1], Step [9901/10950], lr: 4.789954337899544e-06, Loss: 1.1620163917541504
Epoch [1/1], Step [10001/10950], lr: 4.333333333333334e-06, Loss: 0.35599175095558167
Running Evaluation


  0%|          | 0/1322 [00:00<?, ?it/s]

Epoch [1/1], Step [10001/10950], Val_loss: 1.0503
{'exact_match': 63.85998107852412, 'f1': 84.05158351280501}
Epoch [1/1], Step [10101/10950], lr: 3.876712328767123e-06, Loss: 0.3511672914028168
Epoch [1/1], Step [10201/10950], lr: 3.4200913242009136e-06, Loss: 0.4240044355392456
Epoch [1/1], Step [10301/10950], lr: 2.963470319634703e-06, Loss: 0.7607419490814209
Epoch [1/1], Step [10401/10950], lr: 2.5068493150684933e-06, Loss: 0.22961491346359253
Epoch [1/1], Step [10501/10950], lr: 2.050228310502283e-06, Loss: 0.3269726634025574
Epoch [1/1], Step [10601/10950], lr: 1.593607305936073e-06, Loss: 0.43342649936676025
Epoch [1/1], Step [10701/10950], lr: 1.1369863013698631e-06, Loss: 0.3753988742828369
Epoch [1/1], Step [10801/10950], lr: 6.80365296803653e-07, Loss: 0.4571496546268463
Epoch [1/1], Step [10901/10950], lr: 2.2374429223744292e-07, Loss: 0.5795358419418335


In [ ]:
q_n_a(dataset_sqaud, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input:
 Given the following:
An apoplectic stroke deprived him of his speech, and he died shortly afterwards at 2:45 a.m. on 18 February 1546, aged 62, in Eisleben, the city of his birth. He was buried in the Castle Church in Wittenberg, beneath the pulpit. The funeral was held by his friends Johannes Bugenhagen and Philipp Melanchthon. A year later, troops of Luther's adversary Charles V, Holy Roman Emperor entered the town, but were ordered by Charles not to disturb the grave..
Answer the following:
Who performed the funeral for Martin Luther?.

Predicted Answer: ['Charles V']
Reference Answers: {'text': ['Johannes Bugenhagen and Philipp Melanchthon', 'Johannes Bugenhagen and Philipp Melanchthon', 'Johannes Bugenhagen and Philipp Melanchthon'], 'answer_start': [265, 265, 265]}


Spacy based Sementic Similarity: 0.2803806050489905

Bert Scores:


In [ ]:
# Training using HuggingFace Trainer class.
model_id="google/flan-t5-small"
dataset_id="squad"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./qa_finetuned_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=1,
    max_steps=5000,
    # logging & evaluation strategies
    logging_dir=r"./qa_finetuned_model/logs",
    logging_strategy="steps",
    logging_steps=5,
    evaluation_strategy="no",
    save_strategy="no",
    save_total_limit=2,
    load_best_model_at_end=True,
    # report_to="wandb",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"]
)
trainer.train()

Step,Training Loss
5,0.644700
10,0.686300
15,0.541500
20,0.508700
25,0.536000
30,0.409500
35,0.496700
40,0.642300
45,0.518100
50,0.569500


TrainOutput(global_step=5000, training_loss=0.5080767343759537, metrics={'train_runtime': 1668.0729, 'train_samples_per_second': 23.98, 'train_steps_per_second': 2.997, 'total_flos': 7435457533968384.0, 'train_loss': 0.5080767343759537, 'epoch': 0.46})

In [ ]:
evaluate_model()

Running Evaluation


  0%|          | 0/1322 [00:00<?, ?it/s]

Epoch [1/1], Step [10951/10950], Val_loss: 1.3660
{'exact_match': 63.28287606433302, 'f1': 83.74454674657792}
